In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
tf.set_random_seed(777)  # for reproducibility

#f_data = np.loadtxt('crack_or_cicle_training4.csv', delimiter=',', dtype=np.float32)
#f_test = np.loadtxt('crack_or_cicle_test.csv', delimiter=',', dtype=np.float32)

case1_circle = np.loadtxt('circle_only_traing.csv', delimiter=',', dtype=np.float32)
case1_circle_test = np.loadtxt('circle_only_test.csv', delimiter=',', dtype=np.float32)
case1_circle_traingtest = np.loadtxt('circle_only_traingtest.csv', delimiter=',', dtype=np.float32)

case2_crack = np.loadtxt('crack_only_traing.csv', delimiter=',', dtype=np.float32)
case2_crack_test = np.loadtxt('crack_only_test.csv', delimiter=',', dtype=np.float32)
case2_crack_traingtest = np.loadtxt('crack_only_traingtest.csv', delimiter=',', dtype=np.float32)

case3_training = np.loadtxt('crack_or_cicle_trainingset.csv', delimiter=',', dtype=np.float32)
case3_test_cicle = np.loadtxt('crack_or_cicle_circle.csv', delimiter=',', dtype=np.float32)
case3_test_crack = np.loadtxt('crack_or_cicle_crack.csv', delimiter=',', dtype=np.float32)
case3_test_elipse = np.loadtxt('crack_or_cicle_elipse.csv', delimiter=',', dtype=np.float32)

f_test = np.loadtxt('crack_or_cicle_test.csv', delimiter=',', dtype=np.float32)


def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)

In [2]:
case1_circle = MinMaxScaler(case1_circle)
case1_circle_test = MinMaxScaler(case1_circle_test)
case1_circle_traingtest = MinMaxScaler(case1_circle_traingtest)

case2_crack = MinMaxScaler(case2_crack)
case2_crack_test = MinMaxScaler(case2_crack_test)
case2_crack_traingtest = MinMaxScaler(case2_crack_traingtest)

case3_training = MinMaxScaler(case3_training)
case3_test_cicle = MinMaxScaler(case3_test_cicle)
case3_test_crack = MinMaxScaler(case3_test_crack)
case3_test_elipse = MinMaxScaler(case3_test_elipse)


In [3]:
x_data_case1 = case1_circle[:, 0:-1]
y_data_case1 = case1_circle[:, [-1]]

x_test_case1 = case1_circle_traingtest[:, 0:-1]
y_test_case1 = case1_circle_traingtest[:, [-1]]
x_test2_case1 = case1_circle_test[:, 0:-1]
y_test2_case1 = case1_circle_test[:, [-1]]
##############
x_data_case2 = case2_crack[:, 0:-1]
y_data_case2 =case2_crack[:, [-1]]

x_test_case2 = case2_crack_traingtest[:, 0:-1]
y_test_case2 = case2_crack_traingtest[:, [-1]]
x_test2_case2 = case2_crack_test[:, 0:-1]
y_test2_case2 = case2_crack_test[:, [-1]]
##############
x_data_case3 = case3_training[:, 0:-1]
y_data_case3 = case3_training[:, [-1]]

x_test_case3 = case3_test_cicle[:, 0:-1]
y_test_case3 = case3_test_cicle[:, [-1]]
x_test2_case3 = case3_test_crack[:, 0:-1]
y_test2_case3 = case3_test_crack[:, [-1]]
x_test3_case3 = case3_test_elipse[:, 0:-1]
y_test3_case3 = case3_test_elipse[:, [-1]]
##############
#y_test = f_test[:, [-1]]


In [4]:
X = tf.placeholder(tf.float32, shape=[None, 2])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W1 = tf.Variable(tf.random_normal([2,6]), name='weight1')
b1 = tf.Variable(tf.random_normal([6]), name='bias1')
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([6,6]), name='weight2')
b2 = tf.Variable(tf.random_normal([6]), name='bias2')
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([6,4]), name='weight3')
b3 = tf.Variable(tf.random_normal([4]), name='bias3')
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)

W4 = tf.Variable(tf.random_normal([4,1]), name='weight4')
b4 = tf.Variable(tf.random_normal([1]), name='bias4')
hypothesis = tf.matmul(layer3, W4) + b4

In [5]:
learning_rate = [0.001, 0.01]
training_step = [1000, 2000, 10000, 50000, 100000]

In [6]:
for learning in learning_rate:
    print(learning)
    
for training in training_step:
    print(training)


0.001
0.01
1000
2000
10000
50000
100000


In [7]:
# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

cost_history = np.empty(shape=[1],dtype=float)

for learning in learning_rate:

    optimizer = tf.train.AdamOptimizer(learning_rate=learning).minimize(cost)
    
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    for training in training_step:
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        
        print("---------------------------")
        print("--[Learning Rate = ", learning,"]--")
        print("--[  Iteration   = ", training,"  ]--")
        print("---------------------------")
        
        for epoch in range(training+1):
            cost_val, hy_val, _ = sess.run([cost, hypothesis,optimizer],feed_dict={X:x_data_case1,Y:y_data_case1})
           # cost_history = np.append(cost_history,sess.run(cost,feed_dict={X: x_data,Y: y_data}))
            if training > 1000:
                if epoch % 1000 ==0:
                    print("Step: ", epoch, "Cost: ", cost_val)
            else:
                if epoch % 100 == 0:
                    print("Step: ", epoch, "Cost: ", cost_val)
            if training == epoch:
                print("---------------------------")
                print("Learning finished")
                print("---------------------------")
                print("case1_circle_traingtest start")
                print("---------------------------")
                test_val, test_hy = sess.run([cost,hypothesis], feed_dict={X:x_test_case1, Y: y_test_case1})
                print("Test error", test_val, "\n" "Test Prediction" "\n", test_hy )
                print("---------------------------")
                print("case1_circle_test start")
                print("---------------------------")
                test_val, test_hy = sess.run([cost,hypothesis], feed_dict={X:x_test2_case1, Y: y_test2_case1})
                print("Test error", test_val, "\n" "Test Prediction" "\n", test_hy )
             


---------------------------
--[Learning Rate =  0.001 ]--
--[  Iteration   =  1000   ]--
---------------------------
Step:  0 Cost:  238.247
Step:  100 Cost:  86.6486
Step:  200 Cost:  37.4764
Step:  300 Cost:  20.0086
Step:  400 Cost:  10.8155
Step:  500 Cost:  5.54892
Step:  600 Cost:  2.54454
Step:  700 Cost:  0.985843
Step:  800 Cost:  0.313994
Step:  900 Cost:  0.08859
Step:  1000 Cost:  0.0298885
---------------------------
Learning finished
---------------------------
case1_circle_traingtest start
---------------------------
Test error 0.0294469 
Test Prediction
 [[ 0.14282419]
 [ 0.61413646]
 [ 0.20453477]
 [ 0.2223044 ]
 [ 0.47152546]
 [ 0.08593552]
 [ 0.04999086]
 [ 0.06073932]
 [ 0.17767519]
 [ 0.4100149 ]
 [ 0.2657252 ]
 [ 0.1735948 ]
 [ 0.17116289]
 [ 0.38405302]
 [ 0.34296325]
 [ 0.31245735]
 [ 0.15509015]
 [ 0.33930042]
 [ 0.16023096]
 [ 0.20486879]
 [ 0.27311516]
 [ 0.61438394]
 [ 0.56133533]
 [ 0.15036392]
 [ 0.40134051]
 [ 0.09146215]
 [ 0.09960787]
 [ 0.56922245]
 [ 

Step:  1000 Cost:  0.0298885
Step:  2000 Cost:  0.00265349
Step:  3000 Cost:  0.00152637
Step:  4000 Cost:  0.000982474
Step:  5000 Cost:  0.000798316
Step:  6000 Cost:  0.000747955
Step:  7000 Cost:  0.000699761
Step:  8000 Cost:  0.000630498
Step:  9000 Cost:  0.000539426
Step:  10000 Cost:  0.000442016
Step:  11000 Cost:  0.000369555
Step:  12000 Cost:  0.000352777
Step:  13000 Cost:  0.000352585
Step:  14000 Cost:  0.000352586
Step:  15000 Cost:  0.000352585
Step:  16000 Cost:  0.000352584
Step:  17000 Cost:  0.000352584
Step:  18000 Cost:  0.000352585
Step:  19000 Cost:  0.000352586
Step:  20000 Cost:  0.000352442
Step:  21000 Cost:  0.000352048
Step:  22000 Cost:  0.000352048
Step:  23000 Cost:  0.000352048
Step:  24000 Cost:  0.000352048
Step:  25000 Cost:  0.000352047
Step:  26000 Cost:  0.000352047
Step:  27000 Cost:  0.000352047
Step:  28000 Cost:  0.000352048
Step:  29000 Cost:  0.000352048
Step:  30000 Cost:  0.000352047
Step:  31000 Cost:  0.000352046
Step:  32000 Cost:  0

Step:  85000 Cost:  7.36594e-06
Step:  86000 Cost:  7.36096e-06
Step:  87000 Cost:  7.35623e-06
Step:  88000 Cost:  7.35653e-06
Step:  89000 Cost:  7.59722e-06
Step:  90000 Cost:  7.34239e-06
Step:  91000 Cost:  7.33686e-06
Step:  92000 Cost:  7.33404e-06
Step:  93000 Cost:  7.32603e-06
Step:  94000 Cost:  7.51717e-06
Step:  95000 Cost:  7.31594e-06
Step:  96000 Cost:  7.31698e-06
Step:  97000 Cost:  7.30568e-06
Step:  98000 Cost:  7.30697e-06
Step:  99000 Cost:  7.29238e-06
Step:  100000 Cost:  7.2861e-06
---------------------------
Learning finished
---------------------------
case1_circle_traingtest start
---------------------------
Test error 6.8144e-06 
Test Prediction
 [[  7.16196299e-02]
 [  9.88756359e-01]
 [  1.62728280e-01]
 [  2.05369949e-01]
 [  8.04698169e-01]
 [  1.62673593e-02]
 [  3.04618478e-03]
 [  6.99973106e-03]
 [  1.22256935e-01]
 [  7.28397608e-01]
 [  3.09565485e-01]
 [  1.16328269e-01]
 [  1.12794995e-01]
 [  6.96192682e-01]
 [  5.51331282e-01]
 [  4.46193695e-

Step:  1000 Cost:  0.000380062
Step:  2000 Cost:  0.00030877
---------------------------
Learning finished
---------------------------
case1_circle_traingtest start
---------------------------
Test error 0.000347102 
Test Prediction
 [[  8.50943625e-02]
 [  9.77089942e-01]
 [  1.85516179e-01]
 [  2.14433074e-01]
 [  8.10102284e-01]
 [  2.08020210e-05]
 [ -8.52632523e-03]
 [ -3.75899076e-02]
 [  1.41808033e-01]
 [  7.40878880e-01]
 [  2.85092115e-01]
 [  1.35167897e-01]
 [  1.31210685e-01]
 [  7.11661041e-01]
 [  5.55926859e-01]
 [  4.34178412e-01]
 [  1.05055034e-01]
 [  5.41310847e-01]
 [  1.13420606e-01]
 [  1.86060369e-01]
 [  2.97118664e-01]
 [  9.82213914e-01]
 [  9.11175191e-01]
 [  9.73643363e-02]
 [  7.31116712e-01]
 [  8.27056170e-03]
 [  2.04297006e-02]
 [  9.20050323e-01]
 [ -3.38752568e-02]
 [  2.65284359e-01]
 [  1.38934314e-01]
 [  5.45036793e-03]
 [  4.41830575e-01]
 [ -9.81172919e-03]
 [  8.10788572e-01]
 [ -9.75200534e-03]
 [  4.35039103e-01]
 [ -2.49043107e-03]
 [  8.

Step:  1000 Cost:  0.000380062
Step:  2000 Cost:  0.00030877
Step:  3000 Cost:  0.000244774
Step:  4000 Cost:  0.000164852
Step:  5000 Cost:  8.81434e-05
Step:  6000 Cost:  6.39598e-05
Step:  7000 Cost:  6.2133e-05
Step:  8000 Cost:  6.15508e-05
Step:  9000 Cost:  6.14167e-05
Step:  10000 Cost:  6.11822e-05
Step:  11000 Cost:  6.08986e-05
Step:  12000 Cost:  5.87318e-05
Step:  13000 Cost:  0.00036956
Step:  14000 Cost:  5.58947e-05
Step:  15000 Cost:  5.26575e-05
Step:  16000 Cost:  4.97703e-05
Step:  17000 Cost:  4.15448e-05
Step:  18000 Cost:  3.62707e-05
Step:  19000 Cost:  3.15254e-05
Step:  20000 Cost:  2.33543e-05
Step:  21000 Cost:  2.49004e-05
Step:  22000 Cost:  1.76634e-05
Step:  23000 Cost:  1.23304e-05
Step:  24000 Cost:  2.09564e-05
Step:  25000 Cost:  4.1492e-05
Step:  26000 Cost:  1.98591e-05
Step:  27000 Cost:  1.20771e-05
Step:  28000 Cost:  1.54442e-05
Step:  29000 Cost:  1.15876e-05
Step:  30000 Cost:  1.2184e-05
Step:  31000 Cost:  1.07674e-05
Step:  32000 Cost:  1.

In [8]:
# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

cost_history = np.empty(shape=[1],dtype=float)

for learning in learning_rate:

    optimizer = tf.train.AdamOptimizer(learning_rate=learning).minimize(cost)
    
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    for training in training_step:
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        
        print("---------------------------")
        print("--[Learning Rate = ", learning,"]--")
        print("--[  Iteration   = ", training,"  ]--")
        print("---------------------------")
        
        for epoch in range(training+1):
            cost_val, hy_val, _ = sess.run([cost, hypothesis,optimizer],feed_dict={X:x_data_case2, Y: y_data_case2})
           # cost_history = np.append(cost_history,sess.run(cost,feed_dict={X: x_data,Y: y_data}))
            if training > 1000:
                if epoch % 1000 ==0:
                    print("Step: ", epoch, "Cost: ", cost_val)
            else:
                if epoch % 100 == 0:
                    print("Step: ", epoch, "Cost: ", cost_val)
            if training == epoch:
                print("---------------------------")
                print("Learning finished")
                print("---------------------------")
                print("case2_crack_traingtest start")
                print("---------------------------")
                test_val, test_hy = sess.run([cost,hypothesis], feed_dict={X:x_test_case2, Y: y_test_case2})
                print("Test error", test_val, "\n" "Test Prediction" "\n", test_hy )
                print("---------------------------")
                print("case2_crack_test start")
                print("---------------------------")
                test_val, test_hy = sess.run([cost,hypothesis], feed_dict={X:x_test2_case2, Y: y_test2_case2})
                print("Test error", test_val, "\n" "Test Prediction" "\n", test_hy )
             


---------------------------
--[Learning Rate =  0.001 ]--
--[  Iteration   =  1000   ]--
---------------------------
Step:  0 Cost:  83.3216
Step:  100 Cost:  29.2912
Step:  200 Cost:  14.3834
Step:  300 Cost:  8.06966
Step:  400 Cost:  3.74322
Step:  500 Cost:  0.887446
Step:  600 Cost:  0.516317
Step:  700 Cost:  0.334465
Step:  800 Cost:  0.222822
Step:  900 Cost:  0.14789
Step:  1000 Cost:  0.0959824
---------------------------
Learning finished
---------------------------
case2_crack_traingtest start
---------------------------
Test error 0.103931 
Test Prediction
 [[ 0.43912262]
 [ 0.38828868]
 [ 0.39022905]
 [ 0.38679379]
 [ 0.38033742]
 [ 0.38224095]
 [ 0.3956719 ]
 [ 0.38310367]
 [ 0.39297515]
 [ 0.38495916]
 [ 0.39115602]
 [ 0.39267641]
 [ 0.38206643]
 [ 0.38331789]
 [ 0.37780732]
 [ 0.38271266]
 [ 0.38751096]
 [ 0.38691372]
 [ 0.38577276]
 [ 0.38630277]
 [ 0.42288488]
 [ 0.42982489]
 [ 0.39241368]
 [ 0.39219958]
 [ 0.39131933]
 [ 0.38624853]
 [ 0.37785071]
 [ 0.38876909]
 [ 

---------------------------
--[Learning Rate =  0.001 ]--
--[  Iteration   =  50000   ]--
---------------------------
Step:  0 Cost:  83.3216
Step:  1000 Cost:  0.0959824
Step:  2000 Cost:  0.00368045
Step:  3000 Cost:  0.00341031
Step:  4000 Cost:  0.00298562
Step:  5000 Cost:  0.00232887
Step:  6000 Cost:  0.00151078
Step:  7000 Cost:  0.000804809
Step:  8000 Cost:  0.000240185
Step:  9000 Cost:  0.000136281
Step:  10000 Cost:  8.54017e-05
Step:  11000 Cost:  7.38687e-05
Step:  12000 Cost:  6.9852e-05
Step:  13000 Cost:  6.91817e-05
Step:  14000 Cost:  6.9144e-05
Step:  15000 Cost:  6.9144e-05
Step:  16000 Cost:  6.91438e-05
Step:  17000 Cost:  6.91438e-05
Step:  18000 Cost:  6.91436e-05
Step:  19000 Cost:  6.92242e-05
Step:  20000 Cost:  6.91436e-05
Step:  21000 Cost:  6.91437e-05
Step:  22000 Cost:  6.90468e-05
Step:  23000 Cost:  5.35905e-05
Step:  24000 Cost:  4.26472e-05
Step:  25000 Cost:  3.85351e-05
Step:  26000 Cost:  3.40627e-05
Step:  27000 Cost:  2.71101e-05
Step:  28000 

Step:  66000 Cost:  2.13526e-05
Step:  67000 Cost:  2.13527e-05
Step:  68000 Cost:  2.13526e-05
Step:  69000 Cost:  2.13526e-05
Step:  70000 Cost:  2.13545e-05
Step:  71000 Cost:  2.13525e-05
Step:  72000 Cost:  2.13526e-05
Step:  73000 Cost:  2.13526e-05
Step:  74000 Cost:  2.13526e-05
Step:  75000 Cost:  2.13527e-05
Step:  76000 Cost:  2.13526e-05
Step:  77000 Cost:  2.1354e-05
Step:  78000 Cost:  2.13527e-05
Step:  79000 Cost:  2.13527e-05
Step:  80000 Cost:  2.13527e-05
Step:  81000 Cost:  2.13526e-05
Step:  82000 Cost:  2.13527e-05
Step:  83000 Cost:  2.13526e-05
Step:  84000 Cost:  2.13527e-05
Step:  85000 Cost:  2.1379e-05
Step:  86000 Cost:  2.13526e-05
Step:  87000 Cost:  2.13527e-05
Step:  88000 Cost:  2.9894e-05
Step:  89000 Cost:  2.13526e-05
Step:  90000 Cost:  2.13526e-05
Step:  91000 Cost:  2.13527e-05
Step:  92000 Cost:  2.13527e-05
Step:  93000 Cost:  2.13526e-05
Step:  94000 Cost:  2.13525e-05
Step:  95000 Cost:  2.13527e-05
Step:  96000 Cost:  2.13526e-05
Step:  9700

Step:  1000 Cost:  0.000873273
Step:  2000 Cost:  0.000400686
---------------------------
Learning finished
---------------------------
case2_crack_traingtest start
---------------------------
Test error 0.000374824 
Test Prediction
 [[  1.60205364e-02]
 [  5.72678924e-01]
 [  6.99687362e-01]
 [  4.74838644e-01]
 [  1.64786577e-01]
 [  2.29285210e-01]
 [  2.55730450e-02]
 [  2.58519977e-01]
 [  8.79389405e-01]
 [  3.57046574e-01]
 [  7.60342002e-01]
 [  8.59845400e-01]
 [  2.23373026e-01]
 [  2.65787274e-01]
 [  7.90654719e-02]
 [  2.45272845e-01]
 [  5.21754265e-01]
 [  4.82670039e-01]
 [  4.07990724e-01]
 [  4.42692250e-01]
 [ -2.72773504e-02]
 [ -8.77186656e-03]
 [  8.42652321e-01]
 [  8.28611612e-01]
 [  7.71012664e-01]
 [  4.39141899e-01]
 [  8.05376768e-02]
 [  6.04113936e-01]
 [ -1.74963772e-02]
 [  9.73140895e-02]
 [  1.35921180e-01]
 [  9.32222962e-01]
 [  1.79799110e-01]
 [  3.35998803e-01]
 [  1.95998490e-01]
 [  3.49076062e-01]
 [  6.64031029e-01]
 [  1.06284916e-02]
 [  5.

Step:  1000 Cost:  0.000873273
Step:  2000 Cost:  0.000400686
Step:  3000 Cost:  0.000218446
Step:  4000 Cost:  0.000174751
Step:  5000 Cost:  0.000146497
Step:  6000 Cost:  0.000104461
Step:  7000 Cost:  6.64358e-05
Step:  8000 Cost:  5.42914e-05
Step:  9000 Cost:  5.38546e-05
Step:  10000 Cost:  5.24393e-05
Step:  11000 Cost:  5.10073e-05
Step:  12000 Cost:  4.84752e-05
Step:  13000 Cost:  3.73221e-05
Step:  14000 Cost:  3.5287e-05
Step:  15000 Cost:  0.000153115
Step:  16000 Cost:  3.40609e-05
Step:  17000 Cost:  0.000468383
Step:  18000 Cost:  2.26128e-05
Step:  19000 Cost:  2.08407e-05
Step:  20000 Cost:  1.7877e-05
Step:  21000 Cost:  1.48953e-05
Step:  22000 Cost:  1.34987e-05
Step:  23000 Cost:  1.19353e-05
Step:  24000 Cost:  8.52322e-06
Step:  25000 Cost:  6.85129e-06
Step:  26000 Cost:  6.50455e-06
Step:  27000 Cost:  6.24041e-06
Step:  28000 Cost:  6.15023e-06
Step:  29000 Cost:  6.02449e-06
Step:  30000 Cost:  6.03907e-06
Step:  31000 Cost:  2.66373e-05
Step:  32000 Cost: 

In [10]:
# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

cost_history = np.empty(shape=[1],dtype=float)

for learning in learning_rate:

    optimizer = tf.train.AdamOptimizer(learning_rate=learning).minimize(cost)
    
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    for training in training_step:
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        
        print("---------------------------")
        print("--[Learning Rate = ", learning,"]--")
        print("--[  Iteration   = ", training,"  ]--")
        print("---------------------------")
        
        for epoch in range(training+1):
            cost_val, hy_val, _ = sess.run([cost, hypothesis,optimizer],feed_dict={X:x_data_case2, Y: y_data_case2})
           # cost_history = np.append(cost_history,sess.run(cost,feed_dict={X: x_data,Y: y_data}))
            if training > 1000:
                if epoch % 1000 ==0:
                    print("Step: ", epoch, "Cost: ", cost_val)
            else:
                if epoch % 100 == 0:
                    print("Step: ", epoch, "Cost: ", cost_val)
            if training == epoch:
                print("---------------------------")
                print("Learning finished")
                print("---------------------------")
                print("Case3_Circle_test start")
                print("---------------------------")
                test_val, test_hy = sess.run([cost,hypothesis], feed_dict={X:x_test_case3, Y: y_test_case3})
                print("Test error", test_val, "\n" "Test Prediction" "\n", test_hy )
                print("---------------------------")
                print("Case3_Crack_test start")
                print("---------------------------")
                test_val, test_hy = sess.run([cost,hypothesis], feed_dict={X:x_test2_case3, Y: y_test2_case3})
                print("Test error", test_val, "\n" "Test Prediction" "\n", test_hy )
                print("---------------------------")
                print("Case3_Eclpise_test start")
                print("---------------------------")
                test_val, test_hy = sess.run([cost,hypothesis], feed_dict={X:x_test3_case3, Y: y_test3_case3})
                print("Test error", test_val, "\n" "Test Prediction" "\n", test_hy )
             


---------------------------
--[Learning Rate =  0.001 ]--
--[  Iteration   =  1000   ]--
---------------------------
Step:  0 Cost:  83.3216
Step:  100 Cost:  29.2912
Step:  200 Cost:  14.3834
Step:  300 Cost:  8.06966
Step:  400 Cost:  3.74322
Step:  500 Cost:  0.887446
Step:  600 Cost:  0.516317
Step:  700 Cost:  0.334465
Step:  800 Cost:  0.222822
Step:  900 Cost:  0.14789
Step:  1000 Cost:  0.0959824
---------------------------
Learning finished
---------------------------
Case3_Circle_test start
---------------------------
Test error 0.723833 
Test Prediction
 [[ 0.18927343]
 [-0.30169529]
 [ 0.39913172]
 [ 0.40682131]
 [-0.43768781]
 [ 0.31913716]
 [ 0.18618567]
 [-0.90603393]
 [ 0.44549268]
 [ 0.08020051]
 [-0.36064118]
 [ 0.10151942]
 [-0.7146973 ]
 [ 0.39422506]
 [ 0.42022532]
 [ 0.16069825]
 [-0.40462393]
 [ 0.43807834]
 [-0.01681201]
 [-0.06896226]
 [ 0.13867457]
 [ 0.25442296]
 [-0.54043525]
 [ 0.28794771]
 [-0.2505073 ]
 [ 0.41197664]
 [ 0.10408743]
 [ 0.40042037]
 [-0.377

---------------------------
--[Learning Rate =  0.001 ]--
--[  Iteration   =  10000   ]--
---------------------------
Step:  0 Cost:  83.3216
Step:  1000 Cost:  0.0959824
Step:  2000 Cost:  0.00368045
Step:  3000 Cost:  0.00341031
Step:  4000 Cost:  0.00298562
Step:  5000 Cost:  0.00232887
Step:  6000 Cost:  0.00151078
Step:  7000 Cost:  0.000804809
Step:  8000 Cost:  0.000240185
Step:  9000 Cost:  0.000136281
Step:  10000 Cost:  8.54017e-05
---------------------------
Learning finished
---------------------------
Case3_Circle_test start
---------------------------
Test error 0.289713 
Test Prediction
 [[ 0.56824946]
 [ 1.1887548 ]
 [ 0.15178274]
 [ 0.2438737 ]
 [ 1.28200221]
 [ 0.37734246]
 [ 0.57278895]
 [ 1.60314012]
 [-0.00236361]
 [ 0.72859263]
 [ 1.22917342]
 [ 0.69725299]
 [ 1.47194266]
 [ 0.30624318]
 [ 0.03699438]
 [ 0.6102562 ]
 [ 1.25933075]
 [ 0.00918518]
 [ 0.86769557]
 [ 0.93873835]
 [ 0.64263225]
 [ 0.47247601]
 [ 1.35245442]
 [ 0.42319179]
 [ 1.15365601]
 [ 0.29581535]


---------------------------
--[Learning Rate =  0.001 ]--
--[  Iteration   =  100000   ]--
---------------------------
Step:  0 Cost:  83.3216
Step:  1000 Cost:  0.0959824
Step:  2000 Cost:  0.00368045
Step:  3000 Cost:  0.00341031
Step:  4000 Cost:  0.00298562
Step:  5000 Cost:  0.00232887
Step:  6000 Cost:  0.00151078
Step:  7000 Cost:  0.000804809
Step:  8000 Cost:  0.000240185
Step:  9000 Cost:  0.000136281
Step:  10000 Cost:  8.54017e-05
Step:  11000 Cost:  7.38687e-05
Step:  12000 Cost:  6.9852e-05
Step:  13000 Cost:  6.91817e-05
Step:  14000 Cost:  6.9144e-05
Step:  15000 Cost:  6.9144e-05
Step:  16000 Cost:  6.91438e-05
Step:  17000 Cost:  6.91438e-05
Step:  18000 Cost:  6.91436e-05
Step:  19000 Cost:  6.92242e-05
Step:  20000 Cost:  6.91436e-05
Step:  21000 Cost:  6.91437e-05
Step:  22000 Cost:  6.90468e-05
Step:  23000 Cost:  5.35905e-05
Step:  24000 Cost:  4.26472e-05
Step:  25000 Cost:  3.85351e-05
Step:  26000 Cost:  3.40627e-05
Step:  27000 Cost:  2.71101e-05
Step:  28000

---------------------------
--[Learning Rate =  0.01 ]--
--[  Iteration   =  1000   ]--
---------------------------
Step:  0 Cost:  83.3216
Step:  100 Cost:  0.0649904
Step:  200 Cost:  0.00197461
Step:  300 Cost:  0.00175684
Step:  400 Cost:  0.0015116
Step:  500 Cost:  0.00132232
Step:  600 Cost:  0.00119094
Step:  700 Cost:  0.00108609
Step:  800 Cost:  0.00100801
Step:  900 Cost:  0.000938611
Step:  1000 Cost:  0.000873273
---------------------------
Learning finished
---------------------------
Case3_Circle_test start
---------------------------
Test error 0.185543 
Test Prediction
 [[ 0.65386093]
 [ 0.99329448]
 [ 0.15575311]
 [ 0.24764511]
 [ 1.04434764]
 [ 0.54104686]
 [ 0.65654266]
 [ 1.22017014]
 [ 0.03341913]
 [ 0.74860442]
 [ 1.01542354]
 [ 0.73008621]
 [ 1.14833963]
 [ 0.38336727]
 [-0.03411084]
 [ 0.67868137]
 [ 1.03193462]
 [ 0.01102215]
 [ 0.83258188]
 [ 0.87654674]
 [ 0.6978122 ]
 [ 0.59726965]
 [ 1.08292019]
 [ 0.56814885]
 [ 0.9740783 ]
 [ 0.33912638]
 [ 0.72785568]


---------------------------
--[Learning Rate =  0.01 ]--
--[  Iteration   =  10000   ]--
---------------------------
Step:  0 Cost:  83.3216
Step:  1000 Cost:  0.000873273
Step:  2000 Cost:  0.000400686
Step:  3000 Cost:  0.000218446
Step:  4000 Cost:  0.000174751
Step:  5000 Cost:  0.000146497
Step:  6000 Cost:  0.000104461
Step:  7000 Cost:  6.64358e-05
Step:  8000 Cost:  5.42914e-05
Step:  9000 Cost:  5.38546e-05
Step:  10000 Cost:  5.24393e-05
---------------------------
Learning finished
---------------------------
Case3_Circle_test start
---------------------------
Test error 0.474071 
Test Prediction
 [[ 0.7938838 ]
 [ 1.34897935]
 [ 0.13897172]
 [ 0.24715236]
 [ 1.44463742]
 [ 0.49794891]
 [ 0.79807484]
 [ 1.77407396]
 [ 0.0069544 ]
 [ 0.94194829]
 [ 1.39044213]
 [ 0.91300833]
 [ 1.63948691]
 [ 0.36524644]
 [ 0.02240172]
 [ 0.83267319]
 [ 1.42137957]
 [-0.0035021 ]
 [ 1.07318711]
 [ 1.14189517]
 [ 0.86257041]
 [ 0.66879928]
 [ 1.51690888]
 [ 0.57833004]
 [ 1.31297362]
 [ 0.3267

---------------------------
--[Learning Rate =  0.01 ]--
--[  Iteration   =  100000   ]--
---------------------------
Step:  0 Cost:  83.3216
Step:  1000 Cost:  0.000873273
Step:  2000 Cost:  0.000400686
Step:  3000 Cost:  0.000218446
Step:  4000 Cost:  0.000174751
Step:  5000 Cost:  0.000146497
Step:  6000 Cost:  0.000104461
Step:  7000 Cost:  6.64358e-05
Step:  8000 Cost:  5.42914e-05
Step:  9000 Cost:  5.38546e-05
Step:  10000 Cost:  5.24393e-05
Step:  11000 Cost:  5.10073e-05
Step:  12000 Cost:  4.84752e-05
Step:  13000 Cost:  3.73221e-05
Step:  14000 Cost:  3.5287e-05
Step:  15000 Cost:  0.000153115
Step:  16000 Cost:  3.40609e-05
Step:  17000 Cost:  0.000468383
Step:  18000 Cost:  2.26128e-05
Step:  19000 Cost:  2.08407e-05
Step:  20000 Cost:  1.7877e-05
Step:  21000 Cost:  1.48953e-05
Step:  22000 Cost:  1.34987e-05
Step:  23000 Cost:  1.19353e-05
Step:  24000 Cost:  8.52322e-06
Step:  25000 Cost:  6.85129e-06
Step:  26000 Cost:  6.50455e-06
Step:  27000 Cost:  6.24041e-06
Step: